In [7]:
import tensorflow as tf

EPOCHS = 10
NUM_WORDS = 10000

#모델 정의

In [8]:
class RNN(tf.keras.Model):
    def __init__(self):
        super(RNN,self).__init__()
        #단어를 one hot vector로 표현할거임 실수로 embedding을 사용해서 길이가 16인 feature 벡터로 바꿔줌
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.LSTM(32)#적어 적어~~
        self.dense = tf.keras.layers.Dense(2,activation='softmax')#imdb set 쓸건데, 영화 리뷰가 있고, 긍정적이냐 부정적이냐로 함
        
    def __call__(self,x,training = False,mask = None):
        x=self.emb(x)
        x=self.rnn(x)
        return self.dense(x)

#루프 정의

In [9]:
@tf.function
def train_step(model,inputs,labels,loss_object,optimizer,train_loss,train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(inputs,training = True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)
  
@tf.function
def test_step(model,inputs,labels,loss_object,test_loss,test_accuracy):
  predictions = model(inputs, training = False)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

#데이터셋

In [10]:
imdb = tf.keras.datasets.imdb
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words = NUM_WORDS)

#길이 맞춰서 앞에 padding으로 0을 넣어준다.
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32) 

#학습 환경 정의

In [11]:
model = RNN()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

#학습

In [12]:
for epoch in range(EPOCHS):
  for seqs,labels in train_ds:
    train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_seqs, test_labels in test_ds:
    test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch:1, Loss: 0.5010381937026978, Accuracy: 74.43600463867188, Test Loss: 0.44217270612716675, Test Accuracy: 78.81999969482422
Epoch:2, Loss: 0.36999449133872986, Accuracy: 83.42400360107422, Test Loss: 0.45302993059158325, Test Accuracy: 78.85199737548828
Epoch:3, Loss: 0.312820166349411, Accuracy: 86.59200286865234, Test Loss: 0.46954014897346497, Test Accuracy: 78.46800231933594
Epoch:4, Loss: 0.2569601833820343, Accuracy: 89.3479995727539, Test Loss: 0.521030604839325, Test Accuracy: 77.54399871826172
Epoch:5, Loss: 0.20566490292549133, Accuracy: 91.74800109863281, Test Loss: 0.5926687717437744, Test Accuracy: 77.02400207519531
Epoch:6, Loss: 0.15837830305099487, Accuracy: 93.7760009765625, Test Loss: 0.7286831140518188, Test Accuracy: 75.40399932861328
Epoch:7, Loss: 0.12113245576620102, Accuracy: 95.48799896240234, Test Loss: 0.7515762448310852, Test Accuracy: 75.43199920654297
Epoch:8, Loss: 0.09578651934862137, Accuracy: 96.46400451660156, Test Loss: 0.9483672976493835, Test 